# **[:+:] -- Feature Selection + Nearest Neighbor -- [:+:]**

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
#import seaborn as seab
from itertools import chain, combinations
#from scipy.spatial import minkowski_distance
from numba import jit, cuda
from timeit import default_timer as timer
# from itertools import combinations

# [:::] --- TodDo  ---
# - [x] Find "default Rate" : size(most common class)/size(dataset)
# - [x] Create Minkowski Distance function for two points
# - [x] Find nn of a single data point using only one feature
# - [x] Get rid of spurrious percision
# - [x] For small data, find nn of every data point using one feature + max accuracy
# - [x] For small data, find nn of every data point using all features + max accuracy
# - [x] Complete Forward selection for small datas set
# - [x] GPU Multithreading for Classifier
# - [x] Leave-One-Out Cross Validation
# - [x] Complete Backward Elimination for small data set
# - [x] Forward selection methods for large data set
# - [x] Forward Selection validation for both datasets
# - [x] Data Processing funtion
# - [] Backelimination for large Data
# - [] Data Viz
# - [] Report
# - [] User UI
# - [] Third Feature
# - [] Apend Data Processing function with distance matrix formation
# - [] multithread everything else
# - [] Find more brown m&ms
# - [] Clean up the code

### **[+]** Read and Process Data

In [11]:
#Read data from the text files and set as a pandas data frame
# data = pd.read_csv('CS170_Small_Data__19.txt', sep="  ", engine='python', header=None)
# data = pd.read_csv('CS170_Large_Data__3.txt', sep="  ", engine='python', header=None)



# @jit(target_backend='cuda')
def generateDefaultRate(data):
    classCnt= [0,0]
    for pt in data[0]:
        if pt==1:
            classCnt[0]+= 1
        else:
            classCnt[1]+= 1
    defaultRate= max(classCnt)/(classCnt[0]+classCnt[1])
    return defaultRate


# @jit(target_backend='cuda')
def generateCombos(data):
    features= []
    ftCombos= []
    ftRow= data.shape[0]
    for i in range(1,ftRow):
        features.append(i)
    for ft in features:
        for combo in itertools.combinations(features, ft):
            ftCombos.append(combo)
    return features, ftCombos

    
def processDataFrame(dataFrame, display=False):
    if(display):
        #print all the data:
        #print(lilData)

        #----print just the 'label' column (and indeces):
        print("[:] -- The first 4 entries of the first column: data[colIdx][rowIdx]:")
        print(dataFrame[0][0:5])
        print("Type: " +str(type(dataFrame[0][0])))

        print("\n[:] -- Original Matrix Dimensions:")
        lil_rowXcol= dataFrame.shape
        print(lil_rowXcol)

        print("\n[:] -- Data Head:")
        print(dataFrame.head())

        print("\nMetaData: ")
        dataFrame.describe()

        print('\n[:] -- Occurrence counts of classes:')
        # count occurrences of the class column
        occur = dataFrame.groupby([0]).size()
        # display occurrences of a particular column
        display(occur)
        defaultRate= generateDefaultRate(dataFrame)
        print('\nDefault Rate:' + str(defaultRate))

        print("\n[:] -- Convert to numpy array to enable multithreading later:")
        data= dataFrame.to_numpy()
        print("column size after conversion: " + str(len(data[0])))
        data= dataFrame.transpose()
        print("column size after transposition: " + str(len(data[0])))
        print("Dimensions after transposition: " + str(data.shape))
        print("Type: " +str(type(data[0][0])))
    else:
        data= dataFrame.to_numpy().transpose
        ftCombos= generateCombos(dataFrame)
        features= ftCombos[0]
        combinations= ftCombos[1]

    return data, features, combinations



### **[+]** Minkowski Distance Calculation

In [12]:
#My (from-scratch)Function takes in indices of two data points and exponent p, returns the minkowski distance between them based on the four dimensions and the exponent
@jit(target_backend='cuda')
def minkowski(data,rowIdx1,rowIdx2,ftSet,p):
    # featSet1= []
    # featSet2= []
    sigma= 0
    for ft in ftSet:
        # featSet1.append(data[ft][rowIdx1])
        # featSet2.append(data[ft][rowIdx2])
        sigma+= ((((data[ft][rowIdx1]-data[ft][rowIdx2])**2))**(1/2))  **(p)
        distance= sigma**(1/p)
    return distance
    # return distance, featSet1, featSet2

# d= minkowski(lilData, rowIdx1=1, rowIdx2= 2, ftSet=(1,2,3,4,5,6), p=2)
# print(d)
# minkowski_distance(d[1],d[2],p=2)

### **[:+:]** Forward Selection **+** Backward Elimination

In [14]:

        

    




# takes a single combo of features, classifies each dp by its nearest neighbor, returns a list of classifications for every dp
@jit(target_backend='cuda')
def classifier(data, ftSet):
    rows= data.shape[1]
    nearestNeighbors= np.empty(0)
    for dp in range(0,rows):
        nnIdx= 0
        nearest= 10000
        classification= 0
        for  neighborIdx in range(0,rows):
            if dp != neighborIdx:
                dist= minkowski(data, dp, neighborIdx, ftSet, p=2)
                if dist < nearest:
                    nnIdx= neighborIdx
                    nearest= dist
        classification= data[0][nnIdx]
        nearestNeighbors= np.append(nearestNeighbors, classification)
    return nearestNeighbors


#  Returns the accuracy of the classifier tested with a combination of features (leave-one-out, k=n)
# @jit(target_backend='cuda')
def crossValidate(data, ftSet):
    classes= classifier(data, ftSet)
    classesL= len(classes)
    correct= 0
    for j in range(0,classesL):
        if classes[j] == data[0][j]:
            correct+=1
    accuracy = correct/classesL
    return accuracy


# @jit(target_backend='cuda')
def forwardSelection(data, prevFtSet=np.empty(0),finalAccuracy=0, finalSet=np.empty(0)):
    ftSetLen= len(prevFtSet)
    ftRowLength= data.shape[0]
    if ftSetLen>= ftRowLength-1:
        bestFtSet= finalSet
        print("\n[:+:] ---  Forward Selection Results: The best set of features is " +str(bestFtSet)+ " | Accuracy: %" +str(round(finalAccuracy*100,3))+ "  --- [:+:]\n")
    else:
        print("\n[::] -- Forward Selection Search Tree Level "+str(ftSetLen+1)+ " -- [::]" )
        if ftSetLen==0:
            print("        - Testing without features  | Default Rate: %" + str(round(generateDefaultRate(data)*100,3)))
        bestAcc= 0
        bestFtSubset= np.empty(0)
        for ft in range(1, ftRowLength):
            ftSet= np.empty(0)
            if ft not in prevFtSet:
                ftSet= np.append(prevFtSet, ft).astype(int)
                ftAccuracy= crossValidate(data,ftSet)
                print("        - Testing with features "+ str(ftSet) + " | Accuracy: %" + str(round(ftAccuracy*100,3)))
                if ftAccuracy>bestAcc: 
                    bestAcc= ftAccuracy
                    bestFtSubset= ftSet
        if bestAcc>finalAccuracy:
            finalSet= bestFtSubset
            finalAccuracy= bestAcc
        print("       [+] -- On level "+str(ftSetLen+1)+ ", we add feature subset " + str(bestFtSubset)+" with an accuracy of %" + str(round(bestAcc*100,3)))
        bestFtSet=  forwardSelection(data, bestFtSubset,finalAccuracy, finalSet)
    return bestFtSet



def backwardElimination(data, ftCombos, prevFtSet=np.empty(0), eliminated=np.empty(0), globalAcc=0, globalFts=np.empty(0)):
    #ftCombos= generateCombos(data)[1]
    ftRowL= data.shape[0]
    bestAcc= 0
    bestFts= np.empty(0)
    prevComboL= len(prevFtSet)
    leveL= ftRowL - prevComboL
    elimCandidate= []
    print("\n[::] -- Backward Elimination Search Tree Level "+str(leveL)+ " -- [::]" )
    if(prevComboL==1):
        defaultRate= generateDefaultRate(data)
        print("After eliminating all features, we  und up with a default rate of: " +str(defaultRate))
        print("\n[:+:] ---  Backward Selection Results: The best set of features is " +str(globalFts)+ " | Accuracy: %" +str(round(globalAcc*100,3))+ "  --- [:+:]\n")
        return globalFts, globalAcc
    else:
        if(prevComboL==ftRowL-1):
            print("        - Testing with features "+ str(prevFtSet) + " | Accuracy: %" + str(round(crossValidate(data, prevFtSet )*100,3)))
        for i in range(0, len(ftCombos)):
            ftSet= np.empty(0)
            combo= set(ftCombos[i])
            if (len(combo)==(prevComboL-1)):
                commonElmtCnt= len(combo.intersection(eliminated))
                if(commonElmtCnt==0):
                    ftSet= np.append(ftSet, list(combo)).astype(int)
                    ftAccuracy= crossValidate(data, ftSet )
                    print("        - Testing with features "+ str(ftSet) + " | Accuracy: %" + str(round(ftAccuracy*100,3)))
                    if ftAccuracy>bestAcc:
                        bestAcc= ftAccuracy
                        bestFts= ftSet
                        elimCandidate= list(set(prevFtSet).difference((combo)))[0]
        eliminated= np.append(eliminated, elimCandidate).astype(int)
        print("     [-] -- On level "+str(leveL)+ ", we eliminate feature ["+ str(elimCandidate)+"], and end up with feature set: " + str(bestFts)+" with an accuracy of %" + str(round(bestAcc*100,3)))
        print("                                        [:-:] Eliminated so far:--->"+ str(eliminated))
        if bestAcc>globalAcc:
            globalAcc= bestAcc
            globalFts= bestFts
        return backwardElimination(data, ftCombos, bestFts,eliminated, globalAcc, globalFts)



def exhaustiveSelection(data, ftCombos, limit=3):
    bestAcc= 0
    bestFt= np.empty(0)
    for combo in ftCombos:
        if len(combo)==limit:
           ftSet= np.empty(0)
           ftSet= np.append(ftSet, combo).astype(int)
           ftAccuracy= crossValidate(data, ftSet )
           print("        - Testing with features "+ str(ftSet) + " | Accuracy: %" + str(round(ftAccuracy*100,3)))
           if ftAccuracy>bestAcc:
               bestAcc= ftAccuracy
               bestFt= ftSet
    return bestFt, bestAcc




              
                    


# dataMatrix = processDataFrame(pd.read_csv('CS170_Large_Data__3.txt', sep="  ", engine='python', header=None))
dataMatrix = processDataFrame(pd.read_csv('CS170_Small_Data__19.txt', sep="  ", engine='python', header=None))
data= dataMatrix[0]
ftSet= dataMatrix[1]
ftCombos= dataMatrix[2]




start = timer()
print(forwardSelection(data))
print("Time without GPU:", timer()-start)



start = timer()
print(backwardElimination(data,ftCombos, prevFtSet= np.asarray(ftSet)))
print("Time without GPU:", timer()-start)


# start = timer()
# print(exhaustiveSelection(data, ftCombos))
# print("Time without GPU:", timer()-start)

KeyboardInterrupt: 

# **[:+:] Notes**
-   The best single feature is #5, with an accuracy of 0.856

# Works Cited:


-   https://www.analyticsvidhya.com/blog/2020/02/4-types-of-distance-metrics-in-machine-learning/

-   https://www.geeksforgeeks.org/pandas-groupby-count-occurrences-in-column/
-   https://www.codingem.com/python-how-to-get-all-combinations-of-a-list/
-   https://www.geeksforgeeks.org/running-python-script-on-gpu/
-   https://www.programiz.com/python-programming/methods/set/intersection
-   https://www.youtube.com/watch?v=nh2IGC_kLWw
